# Initialization

This notebook is used to set in place everything that initially could be needed to work later on

## Imports

First let's import the math library numpy and the plotting library matplotlib:

In [9]:
# Loading of standard libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt


Next let's set some aspects of the plotting style:

In [10]:
## Set plots to be inline the notebook
%matplotlib inline 

#%matplotlib qt
matplotlib.rc('font', size=16) # Use big fonts...
plt.rcParams['figure.figsize'] = (12.0, 8.0)    # resize plots

# plt.set_cmap('viridis') # <-- Used for XENON1T, looks really fancy
# plt.close()

Finally some really usefull _imports_:

In [11]:
# OS
import os
import sys

# Extra formats
import h5py
import pandas as pd
from collections import defaultdict

# extra features
from tqdm import tqdm ## Get a nice counter for loops
from sklearn import mixture
from matplotlib.colors import LogNorm

# Root
import ROOT
import root_numpy
from ROOT import TH1D

# PAX examples
from pax import units

# For use of minitrees
import hax

# loading images
from IPython.core.display import Image
from IPython.display import Image
import matplotlib.image as mpimg

# Latex
from IPython.display import display
from IPython.display import Latex
from IPython.display import Math

# Get time:
from time import strftime
import datetime
import time
import datetime

# For fitting:
from multihist import Histdd
from scipy.optimize import curve_fit

## Definitions

In [12]:
#Some functions

def decay(time,n_start,halflife):
    return n_start * np.exp(-np.log(2)*(time/halflife))

def exp_decay(x,p0,p1):
    return np.exp(-x*p0)*p1

def exp_decay_lifetime(x,p0,p1):
    return np.exp(-(x*np.log(2))/p0)*p1

def exp_decay_lifetime_lin(x,p0,p1,p2):
    return np.exp(-(x*np.log(2))/p0)*p1 + p2*x

def exponential_decay(t, tau, A):
    
    return A*np.exp(-t/tau)

def linear_function(x, a, b):
# linear function with:
#    p[0] = a
#    p[1] = b

    return a*x + b

def Rtwo(function,data_frame,parameter_x,parameter_y,popt):
    
    r2 = 1-sum(((data_frame[parameter_y]- 
                 function(data_frame[parameter_x],*popt)))**2)/sum((data_frame[parameter_y]-np.mean(data_frame[parameter_y]))**2)
    return r2

variables =("$t$",'a',"$c$","$d$","$e$")
variables2 =('a','b',"$c$","$d$","$e$")
list_func = [exponential_decay,linear_function]
list_names_func = ['Exponential Decay','Linear function']

# text string
def fit_text_string(function,popt,cov,chisq,dof):
    
    textstring = ''
    
    if function == exponential_decay:
        textstring  = """Fit function: \n $y = a \cdot e^{-x /t }$"""
        for i in range(len(popt)): 
            textstring += """\n %.3s = %.3f +/- %.3f"""%(variables[i],popt[i],cov[i,i]**0.5*max(1,np.sqrt(chisq/dof)))
        #textstring += """\n \n $\chi^2_{red} \ \ \ $  = %.3f"""%(chisq/dof)
    elif function == linear_function:
        textstring  = """Fit function: \n $y = a \cdot x + b$"""
        for i in range(len(popt)): 
            textstring += """\n %.3s = %.4f +/- %.4f"""%(variables2[i],popt[i],cov[i,i]**0.5*max(1,np.sqrt(chisq/dof)))
        #textstring += """\n \n $\chi^2_{red} \ \ \ $  = %.3f"""%(chisq/dof) 
    else:
        textstring ="""Unknown Function"""
    return textstring

# text string
def fit_text_string2(function,popt,cov,chisq,dof):
    
    textstring = ''
    
    if function == exponential_decay:
        textstring  = """Fit function: \n $y = a \cdot e^{-x /t }$"""
        for i in range(len(popt)): 
            textstring += """\n %.3s = %.3f +/- %.3f"""%(variables[i],popt[i],cov[i,i]**0.5*max(1,np.sqrt(chisq/dof)))
        textstring += """\n \n $\chi^2_{red} \ \ \ $  = %.3f"""%(chisq/dof)
    elif function == linear_function:
        textstring  = """Fit function: \n $y = a \cdot x + b$"""
        for i in range(len(popt)): 
            textstring += """\n %.3s = %.4f +/- %.4f"""%(variables2[i],popt[i],cov[i,i]**0.5*max(1,np.sqrt(chisq/dof)))
        textstring += """\n \n $\chi^2_{red} \ \ \ $  = %.3f"""%(chisq/dof) 
    else:
        textstring ="""Unknown Function"""
    return textstring

def fit_text_string3(function,popt,cov,chisq,dof,rtwo):
    
    textstring = ''
    
    if function == exponential_decay:
        textstring  = """Fit function: \n $y = a \cdot e^{-x /t }$"""
        for i in range(len(popt)): 
            textstring += """\n %.3s = %.3f +/- %.3f"""%(variables[i],popt[i],cov[i,i]**0.5*max(1,np.sqrt(chisq/dof)))
        textstring += """\n \n $R^2$ = %.3f """%(rtwo)
    elif function == linear_function:
        textstring  = """Fit function: \n $y = a \cdot x + b$"""
        for i in range(len(popt)): 
            textstring += """\n %.3s = %.4f +/- %.4f"""%(variables2[i],popt[i],cov[i,i]**0.5*max(1,np.sqrt(chisq/dof)))
        textstring += """\n \n $R^2$ = %.3f """%(rtwo)
    else:
        textstring ="""Unknown Function"""
    return textstring

def degrees_of_freedom(x_data,popt):

# Calculate degrees of freedom of a given fit, i.e. the number of points minus the number of free parameters of the 
# fit function
# Parameters: data and popt = optimal parameters from the fit used
# Returns: degrees of freedom

    d = len(x_data) - len(popt)
    return d

# calculate chi2 
def chisqure_ndf(data,fit):
    assert(len(data)==len(fit))
    return np.sum((data-fit)**2/(fit))

#Chi-square: This definition can be cross-check with the package lmfit for Python

def chi_square(function,y_data,x_data,popt):
   
    chi = sum(((y_data-function(x_data,*popt)))**2)
    
    return chi 

#Chi-square: Better approach to define a chi-square, considering sigma and empty bins 

def chisquare(function,y_data,x_data,y_sigma,popt):
    
    #initialize a "empty" vector of zeros with the lenght of y_data, just in case y_sigma is not given
    weight = np.zeros_like(y_data)
    if len(y_sigma) == 0:
        y_sigma == weight
        
        
    for i, nx in enumerate(y_data):

        if nx > 10: 
            weight[i] = np.sqrt(nx)
        else:
            weight[i] =  100000000 #function(x_data[i],*popt) # big number so it ignores lower counting
            
    chi2 = sum(((y_data-function(x_data,*popt)))**2/ weight)
    
    return chi2 

def error(array):
    weight = np.zeros_like(array)
    for i, nx in enumerate(array):
        if nx > 10: 
            weight[i] = np.sqrt(nx)
        else:
            weight[i] =  1
            
    return weight


def draw_box(x, y, **kwargs):
    # Arcane syntax of the week: matplotlib's Rectangle...
    plt.gca().add_patch(matplotlib.patches.Rectangle(
        (x[0], y[0]), x[1] - x[0], y[1] - y[0], facecolor='none', **kwargs))

def box_cut(data,x,y,propname_x,propname_y):
    return data[        
        (data[propname_x] >= x[0]) & (data[propname_x] <= x[1]) & (data[propname_y] >= y[0]) & (data[propname_y] <= y[1])]

def do_cut(cut, name=''):
    """Does a cut, prints out acceptance info"""
    global data
    n_before = len(data)
    data = data[cut]
    print("%s: %d events removed (%0.2f%% passthrough)" % (name, n_before - len(data), len(data)/n_before * 100))
    
def do_cut2(cut, name=''):
    """Does a cut, prints out acceptance info"""
    global data2
    n_before2 = len(data2)
    data2 = data2[cut]
    print("%s: %d events removed (%0.2f%% passthrough)" % (name, n_before2 - len(data2), len(data2)/n_before2 * 100))
    
def do_cut3(cut, name=''):
    """Does a cut, prints out acceptance info"""
    global data3
    n_before3 = len(data3)
    data3 = data3[cut]
    print("%s: %d events removed (%0.2f%% passthrough)" % (name, n_before3 - len(data3), len(data3)/n_before3 * 100))

def do_cut4(cut, name=''):
    """Does a cut, prints out acceptance info"""
    global data4
    n_before4 = len(data4)
    data4 = data4[cut]
    print("%s: %d events removed (%0.2f%% passthrough)" % (name, n_before4 - len(data4), len(data4)/n_before4 * 100))
    
def do_cut5(cut, name=''):
    """Does a cut, prints out acceptance info"""
    global data5
    n_before5 = len(data5)
    data5 = data5[cut]
    print("%s: %d events removed (%0.2f%% passthrough)" % (name, n_before5 - len(data5), len(data5)/n_before5 * 100))
    
GasH = 5 #cm:  z coordinate, top to bottom LXe/GXe interface

def fractionEvents(GasL):
    
    return "At this point there are %d Events in the TPC, from which %d porcent of the events (%d) are in the Top part" % (len(data),(data[-data['z_0_good'] < GasL].count()[0]/len(data))*100,data[-data['z_0_good'] < GasL].count()[0])